This tutorial can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/model_uploads/keras-upload-tutorials).

## Wallaroo Model Upload via the Wallaroo SDK: TensorFlow keras Sequential Single IO

The following tutorial demonstrates how to upload a TensorFlow keras Sequential Single IO model to a Wallaroo instance.

### Tutorial Goals

Demonstrate the following:

* Upload a TensorFlow keras Sequential Single IO to a Wallaroo instance.
* Create a pipeline and add the model as a pipeline step.
* Perform a sample inference.

### Prerequisites

* A Wallaroo version 2023.2.1 or above instance.

### References

* [Wallaroo MLOps API Essentials Guide: Model Upload and Registrations](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-api-guide/wallaroo-mlops-api-essential-guide/wallaroo-mlops-api-essentials-guide-model-uploads/)
* [Wallaroo API Connection Guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-api-guide/wallaroo-mlops-connection-guide/)
* [DNS Integration Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-dns-guide/)

## Tutorial Steps

### Import Libraries

The first step is to import the libraries we'll be using.  These are included by default in the Wallaroo instance's JupyterHub service.

In [1]:
import json
import os
import pickle

import wallaroo
from wallaroo.pipeline   import Pipeline
from wallaroo.deployment_config import DeploymentConfigBuilder
from wallaroo.framework import Framework
from wallaroo.object import EntityNotFoundError

import os
os.environ["MODELS_ENABLED"] = "true"

import pyarrow as pa
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

import datetime

### Open a Connection to Wallaroo

The next step is connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  If logging in externally, update the `wallarooPrefix` and `wallarooSuffix` variables with the proper DNS information.  For more information on Wallaroo DNS settings, see the [Wallaroo DNS Integration Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-dns-guide/).

In [2]:
wl = wallaroo.Client()

### Set Variables and Helper Functions

We'll set the name of our workspace, pipeline, models and files.  Workspace names must be unique across the Wallaroo workspace.  For this, we'll add in a randomly generated 4 characters to the workspace name to prevent collisions with other users' workspaces.  If running this tutorial, we recommend hard coding the workspace name so it will function in the same workspace each time it's run.

We'll set up some helper functions that will either use existing workspaces and pipelines, or create them if they do not already exist.

In [3]:
def get_workspace(name, client):
    workspace = None
    for ws in client.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = client.create_workspace(name)
    return workspace


import string
import random

# make a random 4 character suffix to prevent overwriting other user's workspaces
suffix= ''.join(random.choice(string.ascii_lowercase) for i in range(4))

suffix=''

workspace_name = f'keras-sequential-single-io{suffix}'
pipeline_name = f'keras-sequential-single-io'

model_name = 'keras-sequential-single-io'
model_file_name = 'models/model-auto-conversion_keras_single_io_keras_sequential_model.h5'

### Create Workspace and Pipeline

We will now create the Wallaroo workspace to store our model and set it as the current workspace.  Future commands will default to this workspace for pipeline creation, model uploads, etc.  We'll create our Wallaroo pipeline to deploy our model.

In [4]:
workspace = get_workspace(workspace_name, wl)
wl.set_current_workspace(workspace)

pipeline = wl.build_pipeline(pipeline_name)

### Configure Data Schemas

The following parameters are required for TensorFlow keras models.  Note that while some fields are considered as **optional** for the `upload_model` method, they are required for proper uploading of a TensorFlow Keras model to Wallaroo.

| Parameter | Type | Description |
|---|---|---|
|`name` | `string` (*Required*) | The name of the model.  Model names are unique per workspace.  Models that are uploaded with the same name are assigned as a new **version** of the model. |
|`path` | `string` (*Required*) | The path to the model file being uploaded. 
|`framework` |`string` (*Upload Method Optional, TensorFlow keras model Required*) | Set as the `Framework.KERAS`. |
|`input_schema` | `pyarrow.lib.Schema` (*Upload Method Optional, TensorFlow Keras model Required*) | The input schema in Apache Arrow schema format. |
|`output_schema` | `pyarrow.lib.Schema` (*Upload Method Optional, TensorFlow Keras model Required*) | The output schema in Apache Arrow schema format. |
| `convert_wait` | `bool` (*Upload Method Optional, TensorFlow model Optional*) (*Default: True*) | <ul><li>**True**: Waits in the script for the model conversion completion.</li><li>**False**:  Proceeds with the script without waiting for the model conversion process to display complete. |

Once the upload process starts, the model is containerized by the Wallaroo instance.  This process may take up to 10 minutes.

In [5]:
input_schema = pa.schema([
    pa.field('input', pa.list_(pa.float64(), list_size=10))
])
output_schema = pa.schema([
    pa.field('output', pa.list_(pa.float64(), list_size=32))
])

### Upload Model

The model will be uploaded with the framework set as `Framework.KERAS`.

In [6]:
framework=Framework.KERAS

model = wl.upload_model(model_name, 
                        model_file_name, 
                        framework=framework, 
                        input_schema=input_schema, 
                        output_schema=output_schema)
model

Waiting for model loading - this will take up to 10.0min.
Model is pending loading to a native runtime.......
Model is pending loading to a container runtime.
Model is attempting loading to a container runtime...............................................successful

Ready


Name,keras-sequential-single-io
Version,46a40cac-24de-404c-922e-f7ba6136419c
File Name,model-auto-conversion_keras_single_io_keras_sequential_model.h5
SHA,f7e49538e38bebe066ce8df97bac8be239ae8c7d2733e500c8cd633706ae95a8
Status,ready
Image Path,proxy.replicated.com/proxy/wallaroo/ghcr.io/wallaroolabs/mlflow-deploy:v2023.4.1-4514
Architecture,None
Updated At,2024-15-Feb 20:28:10


In [7]:
model.config().runtime()

'flight'

### Deploy Pipeline

The model is uploaded and ready for use.  We'll add it as a step in our pipeline, then deploy the pipeline.  For this example we're allocated 0.25 cpu and 4 Gi RAM to the pipeline through the pipeline's deployment configuration.

In [8]:
deployment_config = DeploymentConfigBuilder() \
    .cpus(0.25).memory('1Gi') \
    .build()

In [ ]:
# clear the pipeline if used in a previous tutorial
pipeline.undeploy()
pipeline.clear()
pipeline.add_model_step(model)

pipeline.deploy(deployment_config=deployment_config)

In [10]:
pipeline.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.100.1.158',
   'name': 'engine-7cf8d599df-5tkd6',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'keras-sequential-single-io',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'keras-sequential-single-io',
      'version': '46a40cac-24de-404c-922e-f7ba6136419c',
      'sha': 'f7e49538e38bebe066ce8df97bac8be239ae8c7d2733e500c8cd633706ae95a8',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.100.1.157',
   'name': 'engine-lb-dcd9c8cd7-tdv5b',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': [{'ip': '10.100.2.218',
   'name': 'engine-sidekick-keras-sequential-single-io-13-66f7ccb8-dp2l2',
   'status': 'Running',
   'reason': None,
   'details': [],
   'statuses': '\n'}]}

### Run Inference

A sample inference will be run.  First the pandas DataFrame used for the inference is created, then the inference run through the pipeline's `infer` method.

In [11]:
input_data = np.random.rand(10, 10)
mock_dataframe = pd.DataFrame({
    "input": input_data.tolist()
})
mock_dataframe

,input
0,"[0.8464678722700603, 0.004712729033793495, 0.2..."
1,"[0.33516546044403395, 0.5002405914748614, 0.57..."
2,"[0.5166812251109929, 0.3730966067105581, 0.606..."
3,"[0.09040120831712439, 0.24578455913569452, 0.4..."
4,"[0.42772724890984193, 0.4738742002929611, 0.63..."
5,"[0.3517207116450022, 0.16363064332614197, 0.22..."
6,"[0.8389813355923412, 0.09430098323471248, 0.51..."
7,"[0.14887831854864786, 0.9237666630636983, 0.92..."
8,"[0.1965534410534433, 0.19881344315974903, 0.38..."
9,"[0.07447775662589717, 0.24736476983489442, 0.1..."


In [12]:
pipeline.infer(mock_dataframe)

,time,in.input,out.output,anomaly.count
0,2024-02-15 20:33:32.212,"[0.8464678723, 0.004712729, 0.2860953943, 0.34...","[0.013701703, 0.032981858, 0.050229844, 0.0267...",0
1,2024-02-15 20:33:32.212,"[0.3351654604, 0.5002405915, 0.5741851642, 0.4...","[0.02636054, 0.02276153, 0.038808007, 0.023839...",0
2,2024-02-15 20:33:32.212,"[0.5166812251, 0.3730966067, 0.6066198208, 0.9...","[0.027724463, 0.033646047, 0.039318968, 0.0227...",0
3,2024-02-15 20:33:32.212,"[0.0904012083, 0.2457845591, 0.4080044601, 0.1...","[0.02969934, 0.028276617, 0.036782112, 0.03051...",0
4,2024-02-15 20:33:32.212,"[0.4277272489, 0.4738742003, 0.6334509482, 0.2...","[0.031070076, 0.028843837, 0.037906397, 0.0269...",0
5,2024-02-15 20:33:32.212,"[0.3517207116, 0.1636306433, 0.2234592269, 0.1...","[0.021969147, 0.028880928, 0.032175012, 0.0386...",0
6,2024-02-15 20:33:32.212,"[0.8389813356, 0.0943009832, 0.5161445909, 0.3...","[0.017825508, 0.030305669, 0.035085037, 0.0335...",0
7,2024-02-15 20:33:32.212,"[0.1488783185, 0.9237666631, 0.9294735798, 0.4...","[0.048920896, 0.020634957, 0.03702375, 0.02326...",0
8,2024-02-15 20:33:32.212,"[0.1965534411, 0.1988134432, 0.382808345, 0.74...","[0.02632338, 0.033858623, 0.031603366, 0.02763...",0
9,2024-02-15 20:33:32.212,"[0.0744777566, 0.2473647698, 0.1291640678, 0.3...","[0.026177717, 0.03367166, 0.027417077, 0.03656...",0


### Undeploy Pipelines

With the tutorial complete, the pipeline is undeployed to return the resources back to the cluster.

In [13]:
pipeline.undeploy()

Waiting for undeployment - this will take up to 45s ..................................... ok


name,keras-sequential-single-io
created,2024-02-15 20:23:36.681423+00:00
last_updated,2024-02-15 20:28:15.115453+00:00
deployed,False
arch,None
tags,
versions,"2ce1e769-f54a-4170-831d-749876c3e683, f1a51e9d-254e-477a-8fa9-830ec9984ae4"
steps,keras-sequential-single-io
published,False
